Importing Packages And Basic Set up

In [141]:
#.venv/scripts/activate  ; no source. 
import xgboost as xgb
from matplotlib.pyplot import hist
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import pandas as pd

import numpy as np
import pandas as pd

RANDOM_SEED = 23


Importing Datasets

In [142]:
# Importing maindata
file_path = "C://Users/miste/Documents/Causal_ML/"
x = pd.read_stata(file_path + "maindata.dta", convert_categoricals=False)

# Importing laws_csv, cleaning it
laws_csv = pd.read_csv(file_path + "When_Were_Laws.csv")
laws_csv = laws_csv[np.logical_not(np.isnan(laws_csv["FIPS"]))]  # FIPS codes identify states
laws_csv = laws_csv.drop("State_Name", axis=1)  # Dropping as useless
laws_csv = laws_csv.rename({'FIPS': 'stfips'}, axis=1) 

# Merging
merged = pd.merge(laws_csv, x, on='stfips', how='outer')


Cleaning Datasets, only interested in the 1997 states. 

In [143]:
basic_merged = merged.copy()  # To allow for re-running 

# Dropping states who were treated < 97 (i.e. they always had programs)
basic_merged = basic_merged[basic_merged["Year_Implemented"].str.contains("always")==False]  

# Making it so that "never-treated" states are treated at T = infinity
basic_merged = basic_merged.replace("never", "1000000") 
basic_merged["Year_Implemented"] = basic_merged["Year_Implemented"].astype(int) # converting to int

# indicator for if treatment has occured in state i
basic_merged["year_indic"] = (basic_merged["year"] >= basic_merged["Year_Implemented"]) 

# Indicator for if the individual was treated (i.e. under 19 and in a state who added a law)
basic_merged["treatment"] = basic_merged["under19"] * basic_merged["year_indic"]

# Generating list of confounders of interest, these are not necessarily optimal. 
list_of_confounders = [ "fownu18", "fpovcut", "povll", "faminctm1", "a_maritl"] 
list_of_confounders += ["a_hga",  "anykids", "year", "stfips", "disability", "elig"] 

# Dropping years  outside of [1995,2000] 
basic_merged = basic_merged[basic_merged["year"] <= 2000]
basic_merged = basic_merged[basic_merged["year"] >= 1995]

# Subsetting our dataset to only include the columns we want, and dropping all rows with empty entries. 
basic_merged = basic_merged[list_of_confounders + ["treatment", "pubonly", "insured", "privonly", "Year_Implemented"]]
basic_merged = basic_merged.dropna(axis = 0)

# Setting up matrices with confounders and either the treatment or no treatment
# In practice, we only use the former. But the latter is useful for predicting hat(g)
confounders_and_treat = basic_merged[list_of_confounders + ["treatment"]]
confounders_no_treat = basic_merged[list_of_confounders]

# Outcome of interest, this can be set to: [ "pubonly", "insured", "privonly"]
y_var = basic_merged["privonly"]

Fitting Q

In [144]:
# This is probably where my model goes wrong, as we're likely overfitting. Keep stratified k-fold
x_train, x_test, y_train, y_test = train_test_split(confounders_and_treat, y_var, test_size=0.2)

Q = HistGradientBoostingClassifier()
Q.fit(x_train, y_train)
score2 = Q.score(x_test,y_test)
print("Model Score: " + str(score2))   
                          
# Can fit g as well, normally get ~95-99%

Model Score: 0.8135968963606134


Estimating causal effect

In [145]:
#INSERT STUFF HERE! Not sure how to do it right. 